<a href="https://colab.research.google.com/github/Elshan777/Text-Summarization-for-Lecture-Notes/blob/master/Evaluate_Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this notebook we will evaluate the results of the previously generated summaries with different both extractive and abstractive models. For this we will use both manually implemented and other methods like BLANC

In [1]:
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter6=True)
!pip install -U sentence-transformers
!pip install blanc

Cloning into 'notebooks'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 507 (delta 100), reused 100 (delta 87), pack-reused 378
Receiving objects: 100% (507/507), 30.39 MiB | 19.44 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2b1eb9611ddf42711e7c95663116334209ac57b65fe1c10564acd8f03950395a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple,

In [2]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm
import pandas as pd

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
df = pd.read_csv('/content/summaries (2).csv')
df.head()

,Unnamed: 0,content,bart,pegasus,bert_extractive,dist_bert_extractive
0,0,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,NaN,NaN
1,1,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...,duality is generally a concept that you can in...,Today we want to look a bit more into convex o...,Today we want to look a bit more into convex o...
2,2,Welcome back to Pattern Recognition!\nToday we...,Today's topic is the Laplacian Support Vector ...,The Laplacian Support Vector Machine allows yo...,Today we want to talk a bit about the Laplacia...,Today we want to talk a bit about the Laplacia...
3,3,Welcome back to pattern recognition.\nToday we...,Today we want to look a bit more into optimiza...,We've seen the gradient descent methods in the...,Today we want to look a bit more into optimiza...,Today we want to look a bit more into optimiza...
4,4,"The Chrysler Building, the famous art deco New...","The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...,"The Chrysler Building, the famous art deco New...","The Chrysler Building, the famous art deco New..."


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
def cosine_similarity(a, b):
    nominator = np.dot(a, b)
    
    a_norm = np.sqrt(np.sum(a**2))
    b_norm = np.sqrt(np.sum(b**2))
    
    denominator = a_norm * b_norm
    
    cosine_similarity = nominator / denominator
    
    return cosine_similarity

def evaluate(model, paragraph, summary):
  # Get sentence embedding for paragraph
  paragraph_embeddings = model.encode(sent_tokenize(paragraph))

  # Get sentence embedding for summary
  summary_embeddings = model.encode(sent_tokenize(summary))

  result = np.array([])
  # loop through summary sentences and find arg max cosine similarity
  for summary in summary_embeddings:
    max = 0
    for paragraph in paragraph_embeddings:
      similarity = cosine_similarity(summary, paragraph) 
      if  similarity > max:
        max = similarity
    result = np.append(result, max)
  # return the mean
  return np.mean(result)

In [6]:
from blanc import BlancHelp, BlancTune

# blanc_tune = BlancTune(finetune_mask_evenly=False, show_progress_bar=False)
blanc_help = BlancHelp(device='cuda')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
metrics = pd.DataFrame()

### Abstractive

In [8]:
metrics['bart_score'] = df[['content', 'bart']].apply(lambda x: evaluate(model, x['content'], x['bart']), axis=1)
metrics['pegasus_score'] = df[['content', 'pegasus']].apply(lambda x: evaluate(model, x['content'], x['pegasus']), axis=1)

In [9]:
df['pegasus_blanc'] = df[['content', 'pegasus']].apply(lambda x: blanc_help.eval_once(x['content'], x['pegasus']), axis=1)
df['bart_blanc'] = df[['content', 'bart']].apply(lambda x: blanc_help.eval_once(x['content'], x['bart']), axis=1)

100%|██████████| 186/186 [00:03<00:00, 61.75it/s]


In [14]:
df.dropna(inplace=True)

### Extractive

In [15]:
metrics['bert_extractive_blanc'] = df[['content', 'bert_extractive']].apply(lambda x: blanc_help.eval_once( x['content'], x['bert_extractive']), axis=1)
metrics['dist_bert_extractive_blanc'] = df[['content', 'dist_bert_extractive']].apply(lambda x: blanc_help.eval_once( x['content'], x['dist_bert_extractive']), axis=1)

100%|██████████| 186/186 [00:03<00:00, 48.23it/s]


In [ ]:
# metrics['bert_extractive_score'] = df[['content', 'bert_extractive']].apply(lambda x: evaluate(model, x['content'], x['bert_extractive']), axis=1)
# metrics['dist_bert_extractive_score'] = df[['content', 'dist_bert_extractive']].apply(lambda x: evaluate(model, x['content'], x['dist_bert_extractive']), axis=1)

In [16]:
metrics.head()

,bart_score,pegasus_score,bert_extractive_blanc,dist_bert_extractive_blanc
0,0.380143,0.377958,NaN,NaN
1,0.937891,0.889664,0.278831,0.300158
2,0.820286,0.793534,0.283276,0.298635
3,0.811754,0.908299,0.319444,0.345486
4,0.965930,0.810819,0.165493,0.169014


In [17]:
metrics.describe()

,bart_score,pegasus_score,bert_extractive_blanc,dist_bert_extractive_blanc
count,13.000000,13.000000,12.000000,12.000000
mean,0.825271,0.803675,0.242047,0.263073
std,0.146658,0.135754,0.044548,0.049861
min,0.380143,0.377958,0.165493,0.169014
25%,0.820286,0.793534,0.219552,0.246336
50%,0.832919,0.815136,0.243258,0.261367
75%,0.902592,0.868556,0.275565,0.299016
max,0.965930,0.911712,0.319444,0.345486


In [ ]:
print('BART score ', metrics['bart_score'].mean())
print('Pegasus score', metrics['pegasus_score'].mean())

BART score  0.8820190027001359
Pegasus score 0.8304945679776595


In [22]:
%cd ..
metrics.to_csv('results.csv', index=False)

/content


In [18]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.6 MB/s 


In [19]:
from bert_score import score

In [32]:
def evaluate_bert_score(paragraph, summary):
  # Get sentence embedding for paragraph
  paragraph_sentences = sent_tokenize(paragraph)
  
  # Get sentence embedding for summary
  summary_sentences =sent_tokenize(summary)

  result = np.array([])
  # loop through summary sentences and find arg max cosine similarity
  for summary in summary_sentences:
    max = 0
    for paragraph in paragraph_sentences:
      P, R, F1 = score([summary], [paragraph], lang="en", verbose=True)
      similarity = F1.mean() 
      if  similarity > max:
        max = similarity
    result = np.append(result, max)
  # return the mean
  return np.mean(result)

In [ ]:
metrics['bart_bert_score'] = df[['content', 'bart']].apply(lambda x: evaluate_bert_score(x['content'], x['bart']), axis=1)
metrics['pegasus_bert_score'] = df[['content', 'pegasus']].apply(lambda x: evaluate_bert_score(x['content'], x['pegasus']), axis=1)

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.11 seconds, 8.74 sentences/sec


In [1]:
mertics

NameError: ignored